In [1]:
!pip install wandb
exit()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 15.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.6 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=5fb8716e50afd596f24db9e1a0e7c2f81f1177cba83da75ecdf2544012a1a219
  Stored in directory: /root/.cache/pip/wheels/b7/0a/67/ada2a22079218c75a88361c0782855cc72aebc4d18d0289d05
Successfully built pathtools


In [1]:
import torch.nn as nn
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import os
from torch.utils.data import DataLoader


In [2]:
import sys
import numpy as np
from PIL import Image


class ProgressBar:
    """
    Prints a progress bar to the standard output (very similar to Keras).
    """

    def __init__(self, width=30):
        """
        Parameters
        ----------
        width : int
            The width of the progress bar (in characters)
        """
        self.width = width

    def update(self, max_value, current_value, info):
        """Updates the progress bar with the given information.
        Parameters
        ----------
        max_value : int
            The maximum value of the progress bar
        current_value : int
            The current value of the progress bar
        info : str
            Additional information that will be displayed to the right of the progress bar
        """
        progress = int(round(self.width * current_value / max_value))
        bar = '=' * progress + '.' * (self.width - progress)
        prefix = '{}/{}'.format(current_value, max_value)

        prefix_max_len = len('{}/{}'.format(max_value, max_value))
        buffer = ' ' * (prefix_max_len - len(prefix))

        sys.stdout.write('\r {} {} [{}] - {}'.format(prefix, buffer, bar, info))
        sys.stdout.flush()

    def new_line(self):
        print()


class Cutout(object):
    """
    Implements Cutout regularization as proposed by DeVries and Taylor (2017), https://arxiv.org/pdf/1708.04552.pdf.
    """

    def __init__(self, num_cutouts, size, p=0.5):
        """
        Parameters
        ----------
        num_cutouts : int
            The number of cutouts
        size : int
            The size of the cutout
        p : float (0 <= p <= 1)
            The probability that a cutout is applied (similar to keep_prob for Dropout)
        """
        self.num_cutouts = num_cutouts
        self.size = size
        self.p = p

    def __call__(self, img):

        height, width = img.size

        cutouts = np.ones((height, width))

        if np.random.uniform() < 1 - self.p:
            return img

        for i in range(self.num_cutouts):
            y_center = np.random.randint(0, height)
            x_center = np.random.randint(0, width)

            y1 = np.clip(y_center - self.size // 2, 0, height)
            y2 = np.clip(y_center + self.size // 2, 0, height)
            x1 = np.clip(x_center - self.size // 2, 0, width)
            x2 = np.clip(x_center + self.size // 2, 0, width)

            cutouts[y1:y2, x1:x2] = 0

        cutouts = np.broadcast_to(cutouts, (3, height, width))
        cutouts = np.moveaxis(cutouts, 0, 2)
        img = np.array(img)
        img = img * cutouts
        return Image.fromarray(img.astype('uint8'), 'RGB')


In [3]:
train_transform = transforms.Compose([
            Cutout(num_cutouts=2, size=8, p=0.8),
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])
test_transform = transforms.Compose([transforms.ToTensor(),
                                             transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
                                             ])
train_dataset = datasets.CIFAR10('data/cifar', train=True, download=True, transform=train_transform)

test_dataset = datasets.CIFAR10('data/cifar', train=False, download=True, transform=test_transform)

100%|██████████| 170498071/170498071 [00:04<00:00, 41577408.52it/s]


Extracting data/cifar/cifar-10-python.tar.gz to data/cifar
Files already downloaded and verified


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
def test(model, batch_size=256):
        """Tests the network.
        """
        model.eval()

        data_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

        correct = 0
        total = 0
        with torch.no_grad():
            for i, data in enumerate(data_loader, 0):
                images, labels = data
                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)
                _, predicted = torch.max(outputs, dim=1)
                total += labels.size(0)
                correct += (predicted == labels.flatten()).sum().item()

        model.train()
        return correct / total

In [6]:
import wandb

wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [7]:
sweep_config = {
    'method': 'random'
    }

sweep_config['metric'] = {
    'name': 'accuracy',
    'goal': 'maximize'
    }

sweep_config['parameters'] = {
        'learning_rate': {
            'values': [0.00001, 0.00005, 0.0001, 0.0005, 0.001]
        }, 
        'optimizer': {
            'values': ['adam', 'sgd']
        },
        'batch_size': {
            'values': [128, 256, 512]
        },
        'epochs': {
            'values': [50,75,100,150,200]
        },
        'activation': {
            'values': ['relu', 'sigmoid']
        },
        'fc_size': {
            'values': [1024, 2048]
        },
        'dropout': {
            'values': [0.25, 0.5]
        }
    }


import pprint

pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'accuracy'},
 'parameters': {'activation': {'values': ['relu', 'sigmoid']},
                'batch_size': {'values': [128, 256, 512]},
                'dropout': {'values': [0.25, 0.5]},
                'epochs': {'values': [50, 75, 100, 150, 200]},
                'fc_size': {'values': [1024, 2048]},
                'learning_rate': {'values': [1e-05,
                                             5e-05,
                                             0.0001,
                                             0.0005,
                                             0.001]},
                'optimizer': {'values': ['adam', 'sgd']}}}


In [8]:
def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = torch.optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = torch.optim.Adam(network.parameters(),
                               lr=learning_rate)
    return optimizer

In [9]:
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels,kernel_size=3, stride=1, padding=1):
        super().__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size, stride=stride, padding=padding, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size, stride=1, padding=padding, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = self.shortcut(x)

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out += residual
        out = self.relu(out)

        return out


class ResNet9(nn.Module):
    def __init__(self, activation='relu', fc_size=1024, dropout=0.5):
        super(ResNet9, self).__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(num_features=64, momentum=0.9),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(num_features=128, momentum=0.9),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            BasicBlock(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(num_features=256, momentum=0.9),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(num_features=256, momentum=0.9),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            BasicBlock(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        if activation == "relu":
            self.fc = nn.Sequential(
                nn.Linear(in_features=1024, out_features=fc_size, bias=True),
                nn.Dropout(p=dropout),
                nn.ReLU(inplace=True),
                nn.BatchNorm1d(num_features=fc_size, momentum=0.9),
                nn.Linear(in_features=fc_size, out_features=10, bias=True)
            )
        elif activation == "sigmoid":
            self.fc = nn.Sequential(
                nn.Linear(in_features=1024, out_features=fc_size, bias=True),
                nn.Dropout(p=dropout),
                nn.Sigmoid(),
                nn.BatchNorm1d(num_features=fc_size, momentum=0.9),
                nn.Linear(in_features=fc_size, out_features=10, bias=True)
            )
        

    def forward(self, x):
        out = self.conv(x)
        out = out.view(-1, out.shape[1] * out.shape[2] * out.shape[3])
        out = self.fc(out)
        return out

In [10]:
def train_wb(config=None):
    with wandb.init(config=config):
        config = wandb.config
        model = ResNet9(activation=config.activation, fc_size=config.fc_size, dropout=config.dropout).to(device)
        print(f"Number of trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")
        optimizer = build_optimizer(model, config.optimizer, config.learning_rate)
        model.train()
        criterion = nn.CrossEntropyLoss()
        trainloader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, num_workers=2)
        
        progress_bar = ProgressBar()
        
        for epoch in range(1, config.epochs + 1):
            print('Epoch {}/{}'.format(epoch, config.epochs))
            running_loss = 0
            epoch_correct = 0
            epoch_total = 0
            for i, (inputs, targets) in enumerate(trainloader):

                inputs, targets = inputs.to(device), targets.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward + backward + optimize
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()

                # Update statistics
                running_loss += loss.item()
                _, predicted = outputs.max(1)
                batch_total = targets.size(0)
                batch_correct = predicted.eq(targets).sum().item()


                epoch_total += batch_total
                epoch_correct += batch_correct

                
                info_str = 'Last batch accuracy: {:.4f} - Running epoch accuracy {:.4f}'.\
                            format(batch_correct / batch_total, epoch_correct / epoch_total)
                progress_bar.update(max_value=len(trainloader), current_value=i, info=info_str)

            progress_bar.new_line()
            test_accuracy = test(model)
            print('Test accuracy: {:.2f}% Loss: {:.4f}'.format(test_accuracy*100,running_loss))
            wandb.log({"accuracy": test_accuracy*100, "epoch": epoch})  

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="DL_Mini")
sweep_id = "goj8y2ak"
wandb.agent(sweep_id, train_wb, count = 5)

Create sweep with ID: w0oap0cz
Sweep URL: https://wandb.ai/brijendra/DL_Mini/sweeps/w0oap0cz


wandb: Agent Starting Run: 5l9outlh with config:
wandb: 	activation: sigmoid
wandb: 	batch_size: 512
wandb: 	dropout: 0.25
wandb: 	epochs: 200
wandb: 	fc_size: 2048
wandb: 	learning_rate: 5e-05
wandb: 	optimizer: sgd
wandb: Currently logged in as: bka2022 (brijendra). Use `wandb login --relogin` to force relogin


Number of trainable parameters: 4561482
Epoch 1/200
 97/98  [==============================] - Last batch accuracy: 0.1994 - Running epoch accuracy 0.1547
Test accuracy: 21.84% Loss: 225.7949
Epoch 2/200
 97/98  [==============================] - Last batch accuracy: 0.2381 - Running epoch accuracy 0.2269
Test accuracy: 28.40% Loss: 206.3838
Epoch 3/200
 97/98  [==============================] - Last batch accuracy: 0.2679 - Running epoch accuracy 0.2633
Test accuracy: 31.81% Loss: 197.3934
Epoch 4/200
 97/98  [==============================] - Last batch accuracy: 0.3125 - Running epoch accuracy 0.2921
Test accuracy: 35.03% Loss: 189.7571
Epoch 5/200
 97/98  [==============================] - Last batch accuracy: 0.3423 - Running epoch accuracy 0.3166
Test accuracy: 38.16% Loss: 183.5549
Epoch 6/200
 97/98  [==============================] - Last batch accuracy: 0.3214 - Running epoch accuracy 0.3345
Test accuracy: 39.85% Loss: 178.6510
Epoch 7/200
 97/98  [===========================

accuracy,▁▃▄▄▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇██████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
accuracy,78.63
epoch,200


wandb: Agent Starting Run: l2xxs0or with config:
wandb: 	activation: sigmoid
wandb: 	batch_size: 512
wandb: 	dropout: 0.25
wandb: 	epochs: 100
wandb: 	fc_size: 1024
wandb: 	learning_rate: 1e-05
wandb: 	optimizer: adam


Number of trainable parameters: 3499594
Epoch 1/100
 97/98  [==============================] - Last batch accuracy: 0.3304 - Running epoch accuracy 0.2468
Test accuracy: 36.56% Loss: 200.8350
Epoch 2/100
 97/98  [==============================] - Last batch accuracy: 0.4524 - Running epoch accuracy 0.3673
Test accuracy: 44.75% Loss: 169.4174
Epoch 3/100
 97/98  [==============================] - Last batch accuracy: 0.4494 - Running epoch accuracy 0.4276
Test accuracy: 50.27% Loss: 154.2791
Epoch 4/100
 97/98  [==============================] - Last batch accuracy: 0.4345 - Running epoch accuracy 0.4614
Test accuracy: 53.62% Loss: 144.6988
Epoch 5/100
 97/98  [==============================] - Last batch accuracy: 0.5298 - Running epoch accuracy 0.4862
Test accuracy: 55.88% Loss: 138.5330
Epoch 6/100
 97/98  [==============================] - Last batch accuracy: 0.4911 - Running epoch accuracy 0.5148
Test accuracy: 57.93% Loss: 132.4685
Epoch 7/100
 97/98  [===========================

accuracy,▁▃▄▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
accuracy,81.57
epoch,100


wandb: Agent Starting Run: 38rv1fw1 with config:
wandb: 	activation: relu
wandb: 	batch_size: 256
wandb: 	dropout: 0.25
wandb: 	epochs: 50
wandb: 	fc_size: 1024
wandb: 	learning_rate: 0.001
wandb: 	optimizer: sgd


Number of trainable parameters: 3499594
Epoch 1/50
 195/196  [==============================] - Last batch accuracy: 0.4375 - Running epoch accuracy 0.3754
Test accuracy: 52.44% Loss: 334.7055
Epoch 2/50
 195/196  [==============================] - Last batch accuracy: 0.5625 - Running epoch accuracy 0.5167
Test accuracy: 59.18% Loss: 262.3875
Epoch 3/50
 195/196  [==============================] - Last batch accuracy: 0.6375 - Running epoch accuracy 0.5775
Test accuracy: 63.22% Loss: 230.5200
Epoch 4/50
 195/196  [==============================] - Last batch accuracy: 0.6250 - Running epoch accuracy 0.6274
Test accuracy: 68.83% Loss: 205.2694
Epoch 5/50
 195/196  [==============================] - Last batch accuracy: 0.5625 - Running epoch accuracy 0.6583
Test accuracy: 70.28% Loss: 188.3811
Epoch 6/50
 195/196  [==============================] - Last batch accuracy: 0.5750 - Running epoch accuracy 0.6833
Test accuracy: 71.65% Loss: 175.0738
Epoch 7/50
 195/196  [====================